In [1]:
from aif360.metrics.classification_metric import ClassificationMetric
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import warnings
import joblib

from utils import make_dataset, display_results

warnings.filterwarnings('ignore')

data = pd.read_csv('../data/processed/compas-scores-two-years-processed.csv')

DROP_COLS = ['two_year_recid','compas_score','decile_score','compas_class']
FEATURE_COLS = data.drop(DROP_COLS, axis=1).columns.tolist()

train, test = train_test_split(data, test_size=0.2, random_state=1234)

def collapse_group(x):
    if x!=3:
        return 0
    else:
        return 1

train['race'] = train['race'].apply(collapse_group)
test['race'] = test['race'].apply(collapse_group)

PRIVILEGED_INFO = {'unprivileged_groups':[{'race': 0}],
                   'privileged_groups':[{'race': 1}]
                  }

BIAS_INFO = {'favorable_label':0,
             'unfavorable_label':1,
             'protected_columns':['race'],
            }

ground_truth_train = make_dataset(train[FEATURE_COLS], train['two_year_recid'], **BIAS_INFO, **PRIVILEGED_INFO)
ground_truth_test = make_dataset(test[FEATURE_COLS], test['two_year_recid'], **BIAS_INFO, **PRIVILEGED_INFO)

# Model fairness logger 

With one function call:

```python
log_fairness_classification_metrics(test['two_year_recid'], test['class_predicted'], test[['race']],
                                    favorable_label=0, unfavorable_label=1,
                                    privileged_groups = {'race':[1]}, unprivileged_groups={'race':[0]})
```

You can log all those metrics:

<img src="../images/fairness_metrics.gif" alt="Drawing" style="width: 600px;"/>

Go to [model fairness workshop public project](https://ui.neptune.ml/o/shared/org/model-fairness-workshop/experiments)

In [2]:
pip install neptune-contrib[all]

Note: you may need to restart the kernel to use updated packages.


In [5]:
from aif360.algorithms.inprocessing import AdversarialDebiasing
import tensorflow as tf

import neptune
from neptunecontrib.monitoring.fairness import log_fairness_classification_metrics

neptune.init(api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5tbCIsImFwaV9rZXkiOiJiNzA2YmM4Zi03NmY5LTRjMmUtOTM5ZC00YmEwMzZmOTMyZTQifQ==',
             project_qualified_name='shared/model-fairness-workshop')

sess = tf.Session()

model_params = {'classifier_num_hidden_units':200, 
                'num_epochs':20,
                'batch_size':128,
                'adversary_loss_weight':0.1,
                'debias':True,
                'seed':1234,
               }

neptune.create_experiment(name='2.0-bonus-fairness-logger', tags=['race', 'adversarial_validation_tweaks'], 
                          params=model_params,
                          upload_source_files=['utils.py','2.0-bonus-fairness_logger.ipynb'])

clf = AdversarialDebiasing(scope_name='debiased_clasifier', sess=sess, **PRIVILEGED_INFO, **model_params)

clf.fit(ground_truth_train)
prediction_test = clf.predict(ground_truth_test)

acc = accuracy_score(test['two_year_recid'], prediction_test.labels)

neptune.log_metric('accuracy',acc)
log_fairness_classification_metrics(test['two_year_recid'], prediction_test.labels, prediction_test.labels, test[['race']], 
                                    favorable_label=0, unfavorable_label=1,
                                    privileged_groups = {'race':[1]}, unprivileged_groups={'race':[0]})

neptune.stop()

https://ui.neptune.ml/shared/model-fairness-workshop/e/MOD-66



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





epoch 0; iter: 0; batch classifier loss: 3.311238; batch adversarial loss: 0.606549
epoch 1; iter: 0; batch classifier loss: 0.923884; batch adversarial loss: 0.738882
epoch 2; iter: 0; batch classifier loss: 0.969863; batch adversarial loss: 0.706319
epoch 3; iter: 0; batch classifier loss: 0.855828; batch adversarial loss: 0.662635
epoch 4; iter: 0; bat

In [7]:
sess.close()
tf.reset_default_graph()